In [ ]:
import numpy as np
import pandas as pd
import itertools
import os

from scipy.stats import wilcoxon

floder_results = "C:\\Users\\cosim\\PycharmProjects\\DDFM\\examples\\synthetic\\results_new"

In [ ]:
# function to convert settings to file name
def return_file_name(non_linear=False, factors=1, sample=50, α=0, ρ=0.5, n_vars=10, missings=0):
    this_comb = tuple((factors, sample, missings, n_vars, non_linear, ρ, α))
    return str(this_comb).replace(",", "").replace("(", "").replace(")", "").replace(" ", "_")
# function to compute statistics
def compute_stats(df, mo, stats, round=3):
    series = df[mo].values
    if stats == "median":
        _out = np.median(series)
    elif stats == "mean":
        _out = np.mean(series)
    elif stats == "std":
        _out = np.std(series)
    return np.round(_out,round)

In [ ]:
# compute stats in all files
#list_done_files = os.listdir("./" + floder_results)
list_done_files = os.listdir(floder_results)
dict_results_mean = dict()
dict_results_median = dict()
dict_results_std = dict()
for this_file in list_done_files:
    if this_file[-4:] == ".csv":
        # init dict info
        sum_stat_res_from_csv = {'median': list(), 
                                 'mean': list(), 
                                 'std': list()}
        by_model_sum_stat_res_from_csv = {'dfm': sum_stat_res_from_csv.copy(),
                                          'ddfm': sum_stat_res_from_csv.copy(),
                                          'ddfm code reduced size': sum_stat_res_from_csv.copy(),
                                          'ddfm code reduced size vs linear f': sum_stat_res_from_csv.copy(),
                                          'ddfm nnlin last neurons': sum_stat_res_from_csv.copy(),
                                          'ddfm nnlin code': sum_stat_res_from_csv.copy(),
                                          'ddfm nnlin code vs linear f': sum_stat_res_from_csv.copy(),
                                         }

        # read file
        this_df = pd.read_csv(floder_results + "/" + this_file)
        this_df["dfm"] = np.maximum(this_df["dfm smoothed"].values, this_df["dfm filtered"].values)
        this_df["ddfm"] = this_df["ddfm code"].values
        
        list_models = list(by_model_sum_stat_res_from_csv.keys())
        list_stats = sum_stat_res_from_csv.keys()
        for mo in list_models:
            for stats in list_stats:
                _outt = compute_stats(this_df, mo, stats)
                by_model_sum_stat_res_from_csv[mo][stats] = _outt
        dict_results_mean[this_file[:-4]] = {key: item["mean"] for key, item in by_model_sum_stat_res_from_csv.items()}
        dict_results_median[this_file[:-4]] = {key: item["median"] for key, item in by_model_sum_stat_res_from_csv.items()}
        dict_results_std[this_file[:-4]] = {key: item["std"] for key, item in by_model_sum_stat_res_from_csv.items()}

In [ ]:
writer = pd.ExcelWriter('results_with_ddfm_nonlin_decoder.xlsx',engine='xlsxwriter')   
workbook=writer.book
worksheet=workbook.add_worksheet('Mean')
writer.sheets['Mean'] = worksheet
pd.DataFrame(dict_results_mean).to_excel(writer,sheet_name='Mean')   
worksheet=workbook.add_worksheet('Median')
writer.sheets['Median'] = worksheet
pd.DataFrame(dict_results_median).to_excel(writer,sheet_name='Median')   
worksheet=workbook.add_worksheet('Std')
writer.sheets['Std'] = worksheet
pd.DataFrame(dict_results_std).to_excel(writer,sheet_name='Std')
writer.close()